In [1]:
import sys
sys.path.append('../..')
from utilities.get_data import get_historical_from_db
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
import ta

In [5]:
# --- Load Data ---
pair = "BTC/USDT"
tf = "1M"

df = get_historical_from_db(ccxt.binance(),pair,tf,path="../../utilities/database/exchanges/")
df

,open,high,low,close,volume
date,,,,,
2020-01-01,7195.24,9578.00,6871.04,9352.89,1.691323e+06
2020-02-01,9351.71,10500.00,8445.00,8523.61,1.609726e+06
2020-03-01,8523.61,9188.00,3782.13,6410.44,3.789769e+06
2020-04-01,6412.14,9460.00,6150.11,8620.00,2.528374e+06
2020-05-01,8620.00,10067.00,8117.00,9448.27,2.685340e+06
2020-06-01,9448.27,10380.00,8833.00,9138.55,1.504746e+06
2020-07-01,9138.08,11444.00,8893.03,11335.46,1.507827e+06
2020-08-01,11335.46,12468.00,10518.50,11649.51,1.891193e+06
2020-09-01,11649.51,12050.85,9825.00,10776.59,1.730389e+06


In [6]:
# --- Function definition ---
def custom_grid(first_price, last_order_down = 0.5, last_order_up = 1, down_grid_len=50, up_grid_len=50):
    """Function that create 2 array grid_buy and grid_sell

    Args:
        first_price (float): price at the begining
        last_order_down (float, optional): Percentage of the last grid buy. Defaults to 0.5.
        last_order_up (int, optional): Percentage of the last grid sell. Defaults to 1.
        down_grid_len (int, optional): Initial length of grid buy. Defaults to 50.
        up_grid_len (int, optional): Initial length of grid sell. Defaults to 50.

    Returns:
        array: return 2 array of float for price.
    """    
    down_pct_unity = last_order_down / down_grid_len
    up_pct_unity = last_order_up / up_grid_len

    grid_sell = []
    grid_buy = []

    for i in range(down_grid_len):
        grid_buy.append(first_price - first_price*down_pct_unity*(i+1))

    for i in range(up_grid_len):
        grid_sell.append(first_price + first_price*up_pct_unity*(i+1))

    return grid_buy, grid_sell

In [7]:
grid_buy, grid_sell = custom_grid(5000, last_order_down = 0.35, last_order_up = 2, down_grid_len=10, up_grid_len=20)
grid_buy


[4825.0,
 4650.0,
 4475.0,
 4300.0,
 4125.0,
 3950.0,
 3775.0,
 3600.0,
 3425.0,
 3250.0]

In [11]:
dt = df.copy().loc[:] # Initial test
# dt = df.copy().loc["2021-01-08":] # Test 1
# dt = df.copy().loc["2020-02-10":] #Test 2
# dt = df.copy().loc["2021-11-19 00":"2022-01-20 17"] #Test 3


first_price = dt.iloc[0]['close']

grid_buy, grid_sell = custom_grid(first_price, last_order_down = 0.35, last_order_up = 16, down_grid_len=30, up_grid_len=70) # Initial nice test
# grid_buy, grid_sell = custom_grid(first_price, last_order_down = 0.3, last_order_up = 1, down_grid_len=40, up_grid_len=60) # Initial bad test
# grid_buy, grid_sell = custom_grid(first_price, last_order_down = 0.35, last_order_up = 0.75, down_grid_len=20, up_grid_len=30) # Test 1
# grid_buy, grid_sell = custom_grid(first_price, last_order_down = 0.7, last_order_up = 2.5, down_grid_len=40, up_grid_len=60) #Test 2
# grid_buy, grid_sell = custom_grid(first_price, last_order_down = 0.35, last_order_up = 0.35, down_grid_len=20, up_grid_len=20) #Test 3

# print("###### Grid sell: ", grid_sell)



trade_list = []
grid_buy_to_insert = 0
grid_sell_to_insert = 0

usd = 500
crypto = 500 / first_price

print("Starting price", first_price)
nb_same_index = 0
for index, row in dt.iterrows():

    try:
        if grid_buy_to_insert > 0:
            # print(row)
            grid_buy_diff = (row["open"] - grid_buy[0]) / (grid_buy_to_insert + 1)
            for i in range(grid_buy_to_insert):
                # print("grid buy", grid_buy[0]+grid_buy_diff)
                grid_buy.insert(0, grid_buy[0]+grid_buy_diff)
        
        if grid_sell_to_insert > 0:
            # print(row)
            grid_sell_diff = (grid_sell[0] - row["open"]) / (grid_sell_to_insert + 1)
            for i in range(grid_sell_to_insert):
                # print("grid_sell", grid_sell[0]-grid_sell_diff)
                grid_sell.insert(0, grid_sell[0]-grid_sell_diff)
        
    except:
        pass

    grid_buy_to_insert = 0
    grid_sell_to_insert = 0

    # if len(grid_buy) + len(grid_sell) != 120:
    #     print("error grid")

    if len(grid_buy) == 0 and usd < 0.05 * (crypto * row["open"]):
        print("End of buy grid => reset wallet and grid", index)
        # --- You can change perameters here for end of grid buy ---
        grid_buy, grid_sell = custom_grid(row["open"], last_order_down = 0.3, last_order_up = 1, down_grid_len=40, up_grid_len=60)
        usd = 0.5 * (usd + crypto * row["open"])
        crypto = 0.5 * (usd + crypto * row["open"]) /  row["open"]

    elif len(grid_sell) == 0 and (crypto * row["open"]) < 0.05 * usd:
        print("End of sell grid => reset wallet and grid", index)
        # --- You can change perameters here for end of grid sell ---
        grid_buy, grid_sell = custom_grid(row["open"], last_order_down = 0.3, last_order_up = 1, down_grid_len=40, up_grid_len=60)
        usd = 0.5 * (usd + crypto * row["open"])
        crypto = 0.5 * (usd + crypto * row["open"]) /  row["open"]

    check_same_index = False
    # -- BUY --
    print("###### Grid sell: ", grid_sell)
    if row["high"] > grid_sell[0]:
        try:
            while row["high"] > grid_sell[0]:
                crypt_to_sell = crypto / len(grid_sell)
                crypto -= crypt_to_sell
                usd += (crypt_to_sell * grid_sell[0])
                trade_list.append({
                    "date": index,
                    "side": "Sell",
                    "usd_amount": crypt_to_sell * grid_sell[0],
                    "price": grid_sell[0],
                    "usd": usd,
                    "crypto": crypto,
                    "wallet": usd + crypto*grid_sell[0]
                })
                grid_buy_to_insert += 1
                del grid_sell[0] 
                check_same_index = True
        except:
            print("End of grid sell",row["close"], index)
            pass

    # -- BUY --
    
    if row["low"] < grid_buy[0]:
        try:
            if check_same_index == True:
                nb_same_index += 1
                # print((row["high"]-row["low"])/row["close"])
            while row["low"] < grid_buy[0]:
                buy_usd_amount = usd/len(grid_buy)
                crypto += (buy_usd_amount / grid_buy[0])
                usd -= buy_usd_amount
                trade_list.append({
                    "date": index,
                    "side": "Buy",
                    "usd_amount": buy_usd_amount,
                    "price": grid_buy[0],
                    "usd": usd,
                    "crypto": crypto,
                    "wallet": usd + crypto*grid_buy[0]
                })
                grid_sell_to_insert += 1
                del grid_buy[0]
        except:
            print("End of grid buy",row["close"], index)
            pass


    
print("Number of same index", nb_same_index)

Starting price 9352.89
###### Grid sell:  [11490.693428571427, 13628.496857142856, 15766.300285714286, 17904.103714285713, 20041.90714285714, 22179.71057142857, 24317.514, 26455.317428571427, 28593.120857142854, 30730.92428571428, 32868.72771428571, 35006.531142857144, 37144.33457142857, 39282.138, 41419.94142857143, 43557.744857142854, 45695.548285714285, 47833.35171428571, 49971.15514285714, 52108.958571428564, 54246.761999999995, 56384.565428571426, 58522.36885714285, 60660.17228571428, 62797.97571428571, 64935.779142857136, 67073.58257142856, 69211.386, 71349.18942857142, 73486.99285714285, 75624.79628571428, 77762.59971428571, 79900.40314285713, 82038.20657142857, 84176.01, 86313.81342857142, 88451.61685714286, 90589.42028571428, 92727.2237142857, 94865.02714285713, 97002.83057142857, 99140.63399999999, 101278.43742857141, 103416.24085714285, 105554.04428571428, 107691.8477142857, 109829.65114285714, 111967.45457142856, 114105.25799999999, 116243.06142857143, 118380.86485714285, 1

IndexError: list index out of range

In [17]:
df_trades = pd.DataFrame(trade_list).iloc[:]
df_trades['wallet_ath'] = df_trades['wallet'].cummax()
df_trades['price_ath'] = df_trades['price'].cummax()
df_trades['wallet_drawdown_pct'] = (df_trades['wallet_ath'] - df_trades['wallet']) / df_trades['wallet_ath']
df_trades['price_drawdown_pct'] = (df_trades['price_ath'] - df_trades['price']) / df_trades['price_ath']
max_trades_drawdown = df_trades['wallet_drawdown_pct'].max()
max_price_drawdown = df_trades['price_drawdown_pct'].max()
wallet_perf = (df_trades.iloc[-1]['wallet'] - df_trades.iloc[0]['wallet']) / df_trades.iloc[0]['wallet']
price_perf = (df_trades.iloc[-1]['price'] - df_trades.iloc[0]['price']) / df_trades.iloc[0]['price']
print("Total trades:", len(df_trades))
print("\n--- Wallet ---")
print("Wallet performance: {}%".format(round(wallet_perf*100, 2)))
print("Worst Wallet Drawdown: -{}%".format(round(max_trades_drawdown*100, 2)))
print("\n--- Asset ---")
print("Asset performance: {}%".format(round(price_perf*100, 2)))
print("Worst Asset Drawdown: -{}%".format(round(max_price_drawdown*100, 2)))
df_trades

Total trades: 27419

--- Wallet ---
Wallet performance: 316.93%
Worst Wallet Drawdown: -43.64%

--- Asset ---
Asset performance: 792.12%
Worst Asset Drawdown: -83.92%


,date,side,usd_amount,price,usd,crypto,wallet,wallet_ath,price_ath,wallet_drawdown_pct,price_drawdown_pct
0,2017-08-17 06:35:00,Sell,8.472222,4332.504667,508.472222,0.115375,1008.333333,1008.333333,4332.504667,0.000000,0.000000
1,2017-08-17 07:45:00,Buy,12.401762,4287.484450,496.070461,0.118267,1003.139143,1008.333333,4332.504667,0.005151,0.010391
2,2017-08-17 08:00:00,Sell,8.583151,4354.454667,504.653612,0.116296,1011.059521,1011.059521,4354.454667,0.000000,0.000000
3,2017-08-17 09:30:00,Sell,8.679883,4403.529333,513.333495,0.114325,1016.766711,1016.766711,4403.529333,0.000000,0.000000
4,2017-08-17 11:00:00,Sell,8.819881,4474.554000,522.153376,0.112354,1024.886602,1024.886602,4474.554000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
27414,2022-03-14 04:10:00,Sell,41.152917,38605.600407,2142.549806,0.053299,4200.195680,5275.842623,68828.323167,0.203882,0.439103
27415,2022-03-14 22:45:00,Sell,41.904010,39310.200814,2184.453815,0.052233,4237.750286,5275.842623,68828.323167,0.196763,0.428866
27416,2022-03-14 22:45:00,Sell,42.536373,39903.421629,2226.990188,0.051167,4268.736091,5275.842623,68828.323167,0.190890,0.420247
27417,2022-03-15 01:10:00,Buy,42.826734,39211.520554,2184.163454,0.052259,4233.333458,5275.842623,68828.323167,0.197601,0.430300
